# 双臂TCP并行接收服务器

## 1. 导入库和定义解码器

In [2]:
import socket
import time
import math
import struct
import numpy as np
from datetime import datetime
from threading import Thread, Lock
from scipy.spatial.transform import Rotation as R
from driver.ftservo_controller import ServoController
from ik.robot import create_so101_5dof
from hw_solver_wrapper import create_hw_solver

print("✓ 导入库成功")

✓ 导入库成功


In [13]:
class JoyconDataDecoder:
    """Joycon 数据解码器（匹配 send.py 的编码格式）"""
    
    FRAME_HEADER = b'\xAA\x55'
    FRAME_TAIL = b'\x0D\x0A'
    FRAME_LENGTH = 22
    
    def __init__(self):
        self.buffer = bytearray()
        self.stats = {'valid_frames': 0, 'error_frames': 0, 'total_bytes': 0}
    
    def decode_position(self, data_bytes):
        """解码3字节位置数据：符号位 + 整数部分 + 小数部分"""
        if len(data_bytes) != 3:
            return 0.0
        sign = 1 if data_bytes[0] == 0 else -1
        int_part = data_bytes[1]
        frac_part = data_bytes[2] / 255.0
        return sign * (int_part + frac_part)
    
    def decode_angle(self, data_bytes):
        """解码2字节角度数据：符号位 + 角度值（度）-> 弧度"""
        if len(data_bytes) != 2:
            return 0.0
        sign = 1 if data_bytes[0] == 0 else -1
        degree = data_bytes[1]
        return sign * (degree * math.pi / 180.0)
    
    def decode_buttons(self, button_byte):
        """解码按钮状态（1字节）"""
        button_states = {
            'x': bool(button_byte & (1 << 0)),
            'home': bool(button_byte & (1 << 1)),
            'plus': bool(button_byte & (1 << 2)),
            'minus': bool(button_byte & (1 << 3)),
            'zr': bool(button_byte & (1 << 4)),
            'r': bool(button_byte & (1 << 5)),
            'b': bool(button_byte & (1 << 6))
        }
        return button_states
    
    def decode_frame(self, frame):
        """解码22字节数据帧"""
        if len(frame) != self.FRAME_LENGTH:
            return None
        
        if frame[0:2] != self.FRAME_HEADER or frame[-2:] != self.FRAME_TAIL:
            return None
        
        try:
            pose_data = []
            
            # 解码位置 X, Y, Z (每组3字节)
            for i in range(3):
                start = 3 + i * 3
                val = self.decode_position(frame[start:start+3])
                pose_data.append(val)
            
            # 解码角度 Rx, Ry, Rz (每组2字节)
            for i in range(3):
                start = 12 + i * 2
                val = self.decode_angle(frame[start:start+2])
                pose_data.append(val)
            
            # 解码夹爪 (Index 18)
            gripper = frame[18]
            
            # 解码按钮状态 (Index 19)
            button_states = self.decode_buttons(frame[19])
            
            return pose_data, gripper, button_states
            
        except Exception as e:
            print(f"✗ 解码异常: {e}")
            return None
    
    def find_and_decode(self):
        """从缓冲区查找并解码一帧"""
        while len(self.buffer) >= self.FRAME_LENGTH:
            header_index = self.buffer.find(self.FRAME_HEADER)
            
            if header_index == -1:
                keep = len(self.FRAME_HEADER) - 1
                if len(self.buffer) > keep:
                    self.buffer = self.buffer[-keep:]
                break
            
            if header_index > 0:
                self.buffer = self.buffer[header_index:]
            
            if len(self.buffer) >= self.FRAME_LENGTH:
                frame = self.buffer[:self.FRAME_LENGTH]
                
                if frame[-2:] == self.FRAME_TAIL:
                    self.buffer = self.buffer[self.FRAME_LENGTH:]
                    self.stats['valid_frames'] += 1
                    return self.decode_frame(frame)
                else:
                    self.buffer = self.buffer[2:]
                    self.stats['error_frames'] += 1
            else:
                break
        
        return None
    
    def add_data(self, data):
        """添加接收到的数据到缓冲区"""
        self.buffer.extend(data)
        self.stats['total_bytes'] += len(data)

print("✓ JoyconDataDecoder 类已定义")

✓ JoyconDataDecoder 类已定义


## 2. 定义双臂接收服务器类

In [ ]:
class DualArmReceiver:
    """双臂 TCP 并行接收服务器"""
    
    def __init__(self, 
                 port_right=46387,
                 port_left=45510,
                 servo_port_right="/dev/right_arm",
                 servo_port_left="/dev/left_arm",
                 servo_baudrate=1_000_000,
                 right_config="./driver/right_arm.json",
                 left_config="./driver/left_arm.json",
                 bitstream_path='design_1.bit',
                 solver_base_right=0x40000000,
                 solver_base_left=0x40010000):
        
        print("="*80)
        print("初始化双臂TCP并行接收服务器...")
        print("="*80)
        
        self.port_right = port_right
        self.port_left = port_left
        self.right_config = right_config
        self.left_config = left_config
        self.bitstream_path = bitstream_path
        self.solver_base_right = solver_base_right
        self.solver_base_left = solver_base_left
        self.running = False
        self.lock = Lock()
        
        # 创建右臂资源
        print("\n[右臂] 初始化...")
        self.right_arm = self._create_arm_dict(port_right, servo_port_right, right_config)
        
        # 创建左臂资源
        print("[左臂] 初始化...")
        self.left_arm = self._create_arm_dict(port_left, servo_port_left, left_config)
        
        # 初始化各臂的硬件
        self._init_arm(self.right_arm, "右臂", solver_base_right, 'xf_solver_lm_so101_0')
        self._init_arm(self.left_arm, "左臂", solver_base_left, 'xf_solver_lm_so101_1')
    
    def _create_arm_dict(self, port, servo_port, config_path):
        """创建臂的资源字典"""
        return {
            'port': port,
            'servo_port': servo_port,
            'config_path': config_path,
            'controller': None,
            'robot': None,
            'hw_solver': None,
            'server_socket': None,
            'client_socket': None,
            'client_address': None,
            'decoder': JoyconDataDecoder(),
            'q_current': None,
            'home_pose': {},
            'base_pos': None,
            'base_rpy': None,
            'current_pos': None,
            'current_rpy': None,
            'home_button_pressed': False,
            'frame_count': 0,
            'decoded_count': 0,
            'ik_params': {
                'ilimit': 500,
                'slimit': 50,
                'tol': 1e-3,
                'mask': [1, 1, 1, 0.8, 0.8, 0],
                'k': 0.1,
                'method': 'sugihara'
            },
            'thread': None
        }
    
    def _init_arm(self, arm_dict, arm_name, solver_base, solver_name):
        """初始化单个机械臂的硬件"""
        try:
            # 初始化舵机控制器
            print(f"  🔧 {arm_name} - 初始化舵机控制器 ({arm_dict['servo_port']})...")
            arm_dict['controller'] = ServoController(
                port=arm_dict['servo_port'],
                baudrate=1_000_000,
                config_path=arm_dict['config_path']
            )
            print(f"  ✓ {arm_name} - 舵机控制器就绪")
        except Exception as e:
            print(f"  ⚠ {arm_name} - 舵机控制器初始化失败: {e}")
        
        try:
            # 创建机器人模型
            print(f"  🤖 {arm_name} - 创建机器人模型...")
            arm_dict['robot'] = create_so101_5dof()
            if arm_dict['controller']:
                arm_dict['robot'].set_servo_controller(arm_dict['controller'])
            print(f"  ✓ {arm_name} - 机器人模型就绪")
        except Exception as e:
            print(f"  ⚠ {arm_name} - 机器人模型初始化失败: {e}")
        
        try:
            # 创建硬件求解器
            print(f"  ⚡ {arm_name} - 初始化硬件IK求解器 (0x{solver_base:08X})...")
            arm_dict['hw_solver'] = create_hw_solver(
                arm_dict['robot'],
                bitstream_path=self.bitstream_path,
                base_addr=solver_base,
                solver_name=solver_name
            )
            if arm_dict['hw_solver'] and arm_dict['hw_solver'].hw_available:
                print(f"  ✓ {arm_name} - 硬件求解器就绪")
            else:
                print(f"  ⚠ {arm_name} - 硬件求解器不可用")
        except Exception as e:
            print(f"  ⚠ {arm_name} - 硬件求解器初始化失败: {e}")
        
        # 初始化Home位置
        if arm_dict['robot'] and arm_dict['controller']:
            for name in arm_dict['robot'].joint_names:
                arm_dict['home_pose'][name] = arm_dict['controller'].get_home_position(name)

print("✓ DualArmReceiver 类已定义")

✓ DualArmReceiver 类已定义


In [20]:
# 添加辅助方法到 DualArmReceiver 类

def _start_arm_server(self, arm_dict, arm_name):
    """为单个机械臂启动TCP服务器"""
    try:
        print(f"\n[{arm_name}] 启动TCP服务器...")
        arm_dict['server_socket'] = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        arm_dict['server_socket'].setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
        arm_dict['server_socket'].bind(('0.0.0.0', arm_dict['port']))
        arm_dict['server_socket'].listen(1)
        print(f"[{arm_name}] ✓ TCP服务器已启动，监听端口 {arm_dict['port']}")
        return True
    except Exception as e:
        print(f"[{arm_name}] ✗ 启动服务器失败: {e}")
        return False

def _wait_for_client(self, arm_dict, arm_name):
    """等待单个机械臂的客户端连接"""
    try:
        print(f"[{arm_name}] 等待客户端连接...")
        arm_dict['client_socket'], arm_dict['client_address'] = arm_dict['server_socket'].accept()
        
        # ========== 关键优化：减少 TCP 延迟 ==========
        # 1. 禁用 Nagle 算法，立即发送小包
        arm_dict['client_socket'].setsockopt(socket.IPPROTO_TCP, socket.TCP_NODELAY, 1)
        # 2. 设置较小的接收缓冲区，减少数据堆积（设为 256 字节，约 11 帧）
        arm_dict['client_socket'].setsockopt(socket.SOL_SOCKET, socket.SO_RCVBUF, 256)
        # 3. 设置非阻塞超时，避免卡死
        arm_dict['client_socket'].settimeout(0.1)
        
        print(f"[{arm_name}] ✓ 客户端已连接: {arm_dict['client_address'][0]}:{arm_dict['client_address'][1]}")
        print(f"[{arm_name}] ✓ 已优化 TCP 参数（TCP_NODELAY + 小缓冲区）")
        arm_dict['frame_count'] = 0
        arm_dict['decoded_count'] = 0
        return True
    except Exception as e:
        print(f"[{arm_name}] ✗ 接受连接失败: {e}")
        return False

def _initialize_arm_home(self, arm_dict, arm_name):
    """初始化机械臂到Home位置"""
    if not arm_dict['controller'] or not arm_dict['robot']:
        print(f"[{arm_name}] ⚠ 硬件未初始化，跳过机械臂初始化")
        return False
    
    try:
        print(f"[{arm_name}] 🏠 移动到Home位置...")
        arm_dict['controller'].move_all_home()
        time.sleep(1.5)
        
        arm_dict['q_current'] = arm_dict['robot'].read_joint_angles(
            joint_names=arm_dict['robot'].joint_names,
            verbose=False
        )
        
        T_current = arm_dict['robot'].fkine(arm_dict['q_current'])
        euler = R.from_matrix(T_current[:3, :3]).as_euler('xyz', degrees=False)
        
        print(f"[{arm_name}] 🔍 当前末端位姿:")
        print(f"  位置: x={T_current[0,3]:.4f}, y={T_current[1,3]:.4f}, z={T_current[2,3]:.4f}")
        print(f"  姿态: roll={euler[0]:.4f}, pitch={euler[1]:.4f}, yaw={euler[2]:.4f}")
        
        arm_dict['current_pos'] = T_current[:3, 3]
        arm_dict['current_rpy'] = euler
        arm_dict['base_pos'] = arm_dict['current_pos'].copy()
        arm_dict['base_rpy'] = arm_dict['current_rpy'].copy()
        
        print(f"[{arm_name}] ✓ 机械臂已初始化")
        return True
    except Exception as e:
        print(f"[{arm_name}] ✗ 初始化失败: {e}")
        return False

# 绑定方法到类
DualArmReceiver._start_arm_server = _start_arm_server
DualArmReceiver._wait_for_client = _wait_for_client
DualArmReceiver._initialize_arm_home = _initialize_arm_home

print("✓ 辅助方法已添加（含 TCP 延迟优化）")

✓ 辅助方法已添加（含 TCP 延迟优化）


In [19]:
# 添加数据处理方法

def _handle_arm_data(self, arm_dict, arm_name):
    """单个机械臂的数据接收和处理线程 - 低延迟优化版"""
    buffer_size = 256  # 减小缓冲区，减少延迟
    last_gripper_value = -1
    
    while self.running:
        if not arm_dict['client_socket']:
            if not self._wait_for_client(arm_dict, arm_name):
                time.sleep(1)
                continue
            self._initialize_arm_home(arm_dict, arm_name)
            arm_dict['decoder'] = JoyconDataDecoder()
            last_gripper_value = -1
            print(f"[{arm_name}] ✓ 开始接收数据（低延迟模式）...\n")
        
        try:
            data = arm_dict['client_socket'].recv(buffer_size)
            
            if not data:
                print(f"\n[{arm_name}] ✗ 客户端断开连接")
                arm_dict['client_socket'].close()
                arm_dict['client_socket'] = None
                continue
            
            arm_dict['frame_count'] += 1
            arm_dict['decoder'].add_data(data)
            
            # ========== 关键优化：只处理最新帧，丢弃旧数据 ==========
            latest_result = None
            frame_count = 0
            while True:
                result = arm_dict['decoder'].find_and_decode()
                if result is None:
                    break
                latest_result = result  # 只保留最新的一帧
                frame_count += 1
                arm_dict['decoded_count'] += 1
            
            # 如果丢弃了旧帧，打印提示（调试用）
            if frame_count > 1:
                print(f"[{arm_name}] ⚡ 跳过 {frame_count - 1} 个旧帧，只处理最新帧")
            
            # 只处理最新的一帧
            if latest_result is None:
                continue
            
            pose, gripper, button_states = latest_result
            x, y, z, rx, ry, rz = pose
            
            timestamp = datetime.now().strftime("%H:%M:%S.%f")[:-3]
            
            # 直接使用解码出来的 gripper 值
            gripper_value = gripper
            
            # 夹爪舵机控制 - 只有值变化时才发送
            if arm_dict['controller'] and gripper_value != last_gripper_value:
                try:
                    gripper_cfg = arm_dict['controller'].config.get('gripper')
                    if gripper_cfg:
                        gripper_range_min = gripper_cfg['range_min']
                        gripper_range_max = gripper_cfg['range_max']
                        gripper_range = gripper_range_max - gripper_range_min
                        
                        gripper_servo_pos = int(gripper_range_min + (gripper_value / 255.0) * gripper_range)
                        gripper_servo_pos = max(gripper_range_min, min(gripper_range_max, gripper_servo_pos))
                        
                        arm_dict['controller'].move_servo('gripper', gripper_servo_pos, speed=1000)
                        last_gripper_value = gripper_value
                except Exception as e:
                    print(f"[{timestamp}] [{arm_name}] ⚠ 夹爪控制异常: {e}")
            
            # Home按钮处理
            if button_states.get('home', False) and not arm_dict['home_button_pressed']:
                arm_dict['home_button_pressed'] = True
                print(f"[{timestamp}] [{arm_name}] 🏠 Home键 → 执行回零...")
                self._initialize_arm_home(arm_dict, arm_name)
            elif not button_states.get('home', False):
                arm_dict['home_button_pressed'] = False
            
            # IK求解和运动执行
            if arm_dict['robot'] and arm_dict['q_current'] is not None:
                try:
                    joycon_offset_pos = np.array([x, y, z])
                    joycon_offset_rpy = np.array([rx, ry, rz])
                    
                    target_pos = arm_dict['base_pos'] + joycon_offset_pos
                    target_rpy = arm_dict['base_rpy'] + joycon_offset_rpy
                    
                    T_goal = np.eye(4)
                    T_goal[:3, :3] = R.from_euler('xyz', target_rpy).as_matrix()
                    T_goal[:3, 3] = target_pos
                    
                    target_euler = np.degrees(target_rpy)
                    
                    # 求解IK
                    sol = arm_dict['robot'].ikine_LM(
                        Tep=T_goal,
                        q0=arm_dict['q_current'],
                        ilimit=arm_dict['ik_params']['ilimit'],
                        slimit=arm_dict['ik_params']['slimit'],
                        tol=arm_dict['ik_params']['tol'],
                        mask=arm_dict['ik_params']['mask'],
                        k=arm_dict['ik_params']['k'],
                        method=arm_dict['ik_params']['method']
                    )
                    
                    if sol.success:
                        with self.lock:
                            print(f"[{timestamp}] [{arm_name}] "
                                  f"pos=[{target_pos[0]:+.4f}, {target_pos[1]:+.4f}, {target_pos[2]:+.4f}] m, "
                                  f"rpy=[{target_euler[0]:+7.2f}°, {target_euler[1]:+7.2f}°, {target_euler[2]:+7.2f}°], "
                                  f"gripper={gripper_value}/255 → ✓ IK成功")
                        
                        arm_dict['q_current'] = sol.q.copy()
                        T_actual = arm_dict['robot'].fkine(arm_dict['q_current'])
                        arm_dict['current_pos'] = T_actual[:3, 3]
                        arm_dict['current_rpy'] = R.from_matrix(T_actual[:3, :3]).as_euler('xyz')
                        
                        # 执行运动
                        if arm_dict['controller']:
                            servo_targets = arm_dict['robot'].q_to_servo_targets(
                                q_rad=sol.q,
                                home_pose=arm_dict['home_pose']
                            )
                            for k in list(servo_targets.keys()):
                                servo_targets[k] = arm_dict['controller'].limit_position(k, servo_targets[k])
                            arm_dict['controller'].fast_move_to_pose(servo_targets)
                    else:
                        with self.lock:
                            print(f"[{timestamp}] [{arm_name}] "
                                  f"pos=[{target_pos[0]:+.4f}, {target_pos[1]:+.4f}, {target_pos[2]:+.4f}] m, "
                                  f"rpy=[{target_euler[0]:+7.2f}°, {target_euler[1]:+7.2f}°, {target_euler[2]:+7.2f}°], "
                                  f"gripper={gripper_value}/255 → ✗ IK失败 ({sol.reason})")
                except Exception as e:
                    with self.lock:
                        print(f"[{timestamp}] [{arm_name}] ✗ IK处理异常: {e}")
        
        except socket.timeout:
            # 超时是正常的，继续循环
            continue
        except Exception as e:
            print(f"[{arm_name}] ✗ 数据处理错误: {e}")
            if arm_dict['client_socket']:
                arm_dict['client_socket'].close()
                arm_dict['client_socket'] = None

DualArmReceiver._handle_arm_data = _handle_arm_data

print("✓ 数据处理方法已添加（低延迟优化版：只处理最新帧）")

✓ 数据处理方法已添加（低延迟优化版：只处理最新帧）


In [18]:
# 添加运行和关闭方法

def run(self):
    """启动双臂并行接收服务器"""
    print("\n" + "="*80)
    print(" "*20 + "PYNQ 双臂TCP并行接收服务器")
    print("="*80)
    print(f"右臂监听端口: {self.port_right}")
    print(f"左臂监听端口: {self.port_left}")
    print(f"比特流文件: {self.bitstream_path}")
    print(f"右臂求解器基地址: 0x{self.solver_base_right:08X}")
    print(f"左臂求解器基地址: 0x{self.solver_base_left:08X}")
    print("="*80)
    
    # 启动服务器
    if not self._start_arm_server(self.right_arm, "右臂"):
        return
    if not self._start_arm_server(self.left_arm, "左臂"):
        return
    
    self.running = True
    
    try:
        # 创建并启动接收线程
        self.right_arm['thread'] = Thread(
            target=self._handle_arm_data,
            args=(self.right_arm, "右臂"),
            daemon=False
        )
        self.left_arm['thread'] = Thread(
            target=self._handle_arm_data,
            args=(self.left_arm, "左臂"),
            daemon=False
        )
        
        print("\n启动并行接收线程...")
        self.right_arm['thread'].start()
        self.left_arm['thread'].start()
        
        print("✓ 双臂并行接收服务已启动\n")
        
        # 等待线程完成
        self.right_arm['thread'].join()
        self.left_arm['thread'].join()
    
    except KeyboardInterrupt:
        print("\n\n用户中断程序")
    except Exception as e:
        print(f"\n\n错误: {e}")
        import traceback
        traceback.print_exc()
    finally:
        self.running = False
        self._shutdown()

def _shutdown(self):
    """关闭所有资源并释放端口"""
    print("\n" + "="*80)
    print("正在关闭双臂接收服务...")
    print("="*80)
    
    for arm_dict, arm_name, port in [(self.right_arm, "右臂", self.port_right), 
                                      (self.left_arm, "左臂", self.port_left)]:
        if arm_dict['client_socket']:
            try:
                arm_dict['client_socket'].close()
            except:
                pass
        
        if arm_dict['server_socket']:
            try:
                # 设置SO_REUSEADDR允许端口快速释放
                arm_dict['server_socket'].setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
                arm_dict['server_socket'].close()
                print(f"✓ {arm_name}TCP服务器端口 {port} 已释放")
            except Exception as e:
                print(f"✗ 关闭{arm_name}服务器连接时出错: {e}")
        
        if arm_dict['controller']:
            try:
                arm_dict['controller'].close()
            except:
                pass
        
        print(f"[{arm_name}] 本次连接统计:")
        print(f"  接收数据包: {arm_dict['frame_count']}")
        print(f"  解码成功帧: {arm_dict['decoded_count']}")
        print(f"  解码统计: {arm_dict['decoder'].stats}")
    
    print("="*80)
    print(f"✓ TCP端口已释放: {self.port_right}, {self.port_left}")
    print("✓ 双臂接收服务已关闭")

DualArmReceiver.run = run
DualArmReceiver._shutdown = _shutdown

print("✓ 运行和关闭方法已添加")

✓ 运行和关闭方法已添加


## 3. 创建并运行双臂接收服务器

In [ ]:
# 创建双臂接收服务器实例
print("创建双臂接收服务器实例...\n")

receiver = DualArmReceiver(
    port_right=46398,
    port_left=45511,
    servo_port_right="/dev/right_arm",
    servo_port_left="/dev/left_arm",
    servo_baudrate=1_000_000,
    right_config="./driver/right_arm.json",
    left_config="./driver/left_arm.json",
    bitstream_path='design_1.bit',
    solver_base_right=0x40000000,
    solver_base_left=0x40010000
)

print("\n✓ 双臂接收服务器已创建")

创建双臂接收服务器实例...

初始化双臂TCP并行接收服务器...

[右臂] 初始化...
[左臂] 初始化...
  🔧 右臂 - 初始化舵机控制器 (/dev/right_arm)...
✅ 已加载舵机配置:
  1: shoulder_pan (home_pose=1974, range=797~3151)
  2: shoulder_lift (home_pose=2130, range=946~3314)
  3: elbow_flex (home_pose=1906, range=803~3009)
  4: wrist_flex (home_pose=2043, range=870~3216)
  5: wrist_roll (home_pose=2032, range=120~3945)
  6: gripper (home_pose=1993, range=1993~3507)
  ✓ 右臂 - 舵机控制器就绪
  🤖 右臂 - 创建机器人模型...
  ✓ 右臂 - 机器人模型就绪
  ⚡ 右臂 - 初始化硬件IK求解器 (0x40000000)...
✓ 硬件求解器已初始化 (基址: 0x40000000, 核名: xf_solver_lm_so101_0)
  ✓ 右臂 - 硬件求解器就绪
  🔧 左臂 - 初始化舵机控制器 (/dev/left_arm)...
✅ 已加载舵机配置:
  1: shoulder_pan (home_pose=2070, range=719~3421)
  2: shoulder_lift (home_pose=2062, range=884~3240)
  3: elbow_flex (home_pose=1949, range=852~3047)
  4: wrist_flex (home_pose=2000, range=861~3140)
  5: wrist_roll (home_pose=2088, range=179~3997)
  6: gripper (home_pose=2050, range=2050~3535)
  ✓ 左臂 - 舵机控制器就绪
  🤖 左臂 - 创建机器人模型...
  ✓ 左臂 - 机器人模型就绪
  ⚡ 左臂 - 初始化硬件IK求解器 (0x40010000).

In [22]:
# 启动双臂并行接收服务
# 注意：这会启动两个并行线程，分别监听两个端口接收两个手柄的数据
# 使用 Ctrl+C 可以中断服务

receiver.run()


                    PYNQ 双臂TCP并行接收服务器
右臂监听端口: 46398
左臂监听端口: 45511
比特流文件: design_8.bit
右臂求解器基地址: 0x40000000
左臂求解器基地址: 0x40010000

[右臂] 启动TCP服务器...
[右臂] ✓ TCP服务器已启动，监听端口 46398

[左臂] 启动TCP服务器...
[左臂] ✓ TCP服务器已启动，监听端口 45511

启动并行接收线程...
[右臂] 等待客户端连接...
[左臂] 等待客户端连接...
✓ 双臂并行接收服务已启动

[右臂] ✓ 客户端已连接: 192.168.2.2:39704[左臂] ✓ 客户端已连接: 192.168.2.2:39674
[左臂] ✓ 已优化 TCP 参数（TCP_NODELAY + 小缓冲区）
[左臂] 🏠 移动到Home位置...
🏠 全部舵机同步回中位完成

[右臂] ✓ 已优化 TCP 参数（TCP_NODELAY + 小缓冲区）
[右臂] 🏠 移动到Home位置...
🏠 全部舵机同步回中位完成
[左臂] 🔍 当前末端位姿:[右臂] 🔍 当前末端位姿:
  位置: x=0.3361, y=0.0013, z=0.2341
  姿态: roll=-0.0015, pitch=0.0368, yaw=0.0046
[右臂] ✓ 机械臂已初始化
[右臂] ✓ 开始接收数据（低延迟模式）...


  位置: x=0.3352, y=-0.0004, z=0.2310
  姿态: roll=-0.0061, pitch=0.0537, yaw=-0.0015
[左臂] ✓ 机械臂已初始化
[左臂] ✓ 开始接收数据（低延迟模式）...

[右臂] ⚡ 跳过 8 个旧帧，只处理最新帧
[左臂] ⚡ 跳过 8 个旧帧，只处理最新帧
✅ gripper(6) → 1993
[19:23:26.726] [右臂] pos=[+0.3361, +0.0013, +0.2341] m, rpy=[  +0.91°,   +2.11°,   +0.26°], gripper=0/255 → ✓ IK成功
✅ gripper(6) → 2050
[19:23:26.728] [左臂] pos=[+0.3352, -0.

🚀 Fast move (global) speed mode
🚀 Fast move (global) speed mode
[19:23:39.410] [左臂] pos=[+0.3351, -0.0004, +0.2314] m, rpy=[  -1.35°,   +1.64°,   -0.09°], gripper=0/255 → ✓ IK成功
[19:23:39.431] [右臂] pos=[+0.3369, +0.0013, +0.2312] m, rpy=[  +0.26°,   +1.81°,   +0.26°], gripper=0/255 → ✓ IK成功
🚀 Fast move (global) speed mode
🚀 Fast move (global) speed mode
[19:23:39.512] [左臂] pos=[+0.3351, -0.0004, +0.2314] m, rpy=[  -1.35°,   +1.64°,   -0.09°], gripper=0/255 → ✓ IK成功
[19:23:39.510] [右臂] pos=[+0.3369, +0.0013, +0.2312] m, rpy=[  -0.74°,   +2.81°,  -11.74°], gripper=0/255 → ✓ IK成功
🚀 Fast move (global) speed mode
🚀 Fast move (global) speed mode
[19:23:39.612] [右臂] pos=[+0.3369, +0.0013, +0.2312] m, rpy=[  +4.26°,   +1.81°,   -9.74°], gripper=0/255 → ✓ IK成功
[19:23:39.610] [左臂] pos=[+0.3351, -0.0004, +0.2314] m, rpy=[  -1.35°,   +1.64°,   -0.09°], gripper=0/255 → ✓ IK成功
🚀 Fast move (global) speed mode
🚀 Fast move (global) speed mode
[19:23:39.710] [左臂] pos=[+0.3351, -0.0004, +0.2314] m, rpy=[

🚀 Fast move (global) speed mode
[19:23:42.213] [左臂] pos=[+0.3351, -0.0004, +0.2314] m, rpy=[  -1.35°,   +1.64°,   -0.09°], gripper=0/255 → ✓ IK成功
[19:23:42.235] [右臂] pos=[+0.3369, +0.0013, +0.2312] m, rpy=[  +1.26°,   +2.81°,  -12.74°], gripper=0/255 → ✓ IK成功
🚀 Fast move (global) speed mode
🚀 Fast move (global) speed mode
[19:23:42.313] [左臂] pos=[+0.3351, -0.0004, +0.2314] m, rpy=[  -1.35°,   +1.64°,   -0.09°], gripper=0/255 → ✓ IK成功
[19:23:42.315] [右臂] pos=[+0.3369, +0.0013, +0.2312] m, rpy=[  +1.26°,   +2.81°,  -12.74°], gripper=0/255 → ✓ IK成功
🚀 Fast move (global) speed mode
🚀 Fast move (global) speed mode
[19:23:42.413] [左臂] pos=[+0.3351, -0.0004, +0.2314] m, rpy=[  -1.35°,   +1.64°,   -0.09°], gripper=0/255 → ✓ IK成功
[19:23:42.430] [右臂] pos=[+0.3369, +0.0013, +0.2312] m, rpy=[  +1.26°,   +2.81°,  -12.74°], gripper=0/255 → ✓ IK成功
🚀 Fast move (global) speed mode
🚀 Fast move (global) speed mode
[19:23:42.513] [左臂] pos=[+0.3351, -0.0004, +0.2314] m, rpy=[  -1.35°,   +1.64°,   -0.09°], g

[19:23:45.016] [右臂] pos=[+0.3369, +0.0013, +0.2312] m, rpy=[  +3.26°,  +31.81°,  -11.74°], gripper=0/255 → ✓ IK成功
[19:23:45.019] [左臂] pos=[+0.3351, -0.0004, +0.2314] m, rpy=[  -1.35°,   +1.64°,   -0.09°], gripper=0/255 → ✓ IK成功
🚀 Fast move (global) speed mode
🚀 Fast move (global) speed mode
[19:23:45.119] [右臂] pos=[+0.3369, +0.0013, +0.2312] m, rpy=[  +1.26°,  +32.81°,  -13.74°], gripper=0/255 → ✓ IK成功
[19:23:45.117] [左臂] pos=[+0.3351, -0.0004, +0.2314] m, rpy=[  -1.35°,   +1.64°,   -0.09°], gripper=0/255 → ✓ IK成功
🚀 Fast move (global) speed mode
🚀 Fast move (global) speed mode
[19:23:45.217] [左臂] pos=[+0.3351, -0.0004, +0.2314] m, rpy=[  -1.35°,   +1.64°,   -0.09°], gripper=0/255 → ✓ IK成功
[19:23:45.238] [右臂] pos=[+0.3369, +0.0013, +0.2312] m, rpy=[  -0.74°,  +32.81°,  -17.74°], gripper=0/255 → ✓ IK成功
🚀 Fast move (global) speed mode
🚀 Fast move (global) speed mode
[19:23:45.317] [右臂] pos=[+0.3369, +0.0013, +0.2312] m, rpy=[  -7.74°,  +32.81°,  -20.74°], gripper=0/255 → ✓ IK成功
[19:23:45.

[19:23:47.922] [右臂] pos=[+0.3369, +0.0013, +0.2312] m, rpy=[ -24.74°,  +28.81°,  -21.74°], gripper=0/255 → ✓ IK成功
🚀 Fast move (global) speed mode
🚀 Fast move (global) speed mode
[19:23:48.020] [左臂] pos=[+0.3351, -0.0004, +0.2314] m, rpy=[  -1.35°,   +1.64°,   -0.09°], gripper=0/255 → ✓ IK成功
[19:23:48.022] [右臂] pos=[+0.3369, +0.0013, +0.2312] m, rpy=[ -19.74°,  +29.81°,  -16.74°], gripper=0/255 → ✓ IK成功
🚀 Fast move (global) speed mode
🚀 Fast move (global) speed mode
[19:23:48.120] [右臂] pos=[+0.3369, +0.0013, +0.2312] m, rpy=[ -11.74°,  +31.81°,   -8.74°], gripper=0/255 → ✓ IK成功
[19:23:48.122] [左臂] pos=[+0.3351, -0.0004, +0.2314] m, rpy=[  -1.35°,   +1.64°,   -0.09°], gripper=0/255 → ✓ IK成功
🚀 Fast move (global) speed mode
🚀 Fast move (global) speed mode
[19:23:48.220] [左臂] pos=[+0.3351, -0.0004, +0.2314] m, rpy=[  -1.35°,   +1.64°,   -0.09°], gripper=0/255 → ✓ IK成功
🚀 Fast move (global) speed mode
[19:23:48.237] [右臂] pos=[+0.3369, +0.0013, +0.2312] m, rpy=[  -1.74°,  +32.81°,   -4.74°], g

...

[19:24:43.489] [左臂] pos=[+0.3351, -0.0004, +0.2314] m, rpy=[  +6.65°,   -7.36°,  +22.91°], gripper=0/255 → ✓ IK成功
[19:24:43.506] [右臂] pos=[+0.2977, +0.0562, +0.1881] m, rpy=[  +1.26°,   +2.81°,  -28.74°], gripper=0/255 → ✓ IK成功
🚀 Fast move (global) speed mode
🚀 Fast move (global) speed mode
[19:24:43.589] [左臂] pos=[+0.3351, -0.0004, +0.2314] m, rpy=[  +5.65°,   -8.36°,  +22.91°], gripper=0/255 → ✓ IK成功
[19:24:43.611] [右臂] pos=[+0.2977, +0.0562, +0.1881] m, rpy=[  +1.26°,   +2.81°,  -28.74°], gripper=0/255 → ✓ IK成功
🚀 Fast move (global) speed mode
🚀 Fast move (global) speed mode
[19:24:43.689] [左臂] pos=[+0.3351, -0.0004, +0.2314] m, rpy=[  +5.65°,   -8.36°,  +21.91°], gripper=0/255 → ✓ IK成功
[19:24:43.711] [右臂] pos=[+0.2977, +0.0562, +0.1881] m, rpy=[  +1.26°,   +2.81°,  -28.74°], gripper=0/255 → ✓ IK成功
🚀 Fast move (global) speed mode
🚀 Fast move (global) speed mode
[19:24:43.790] [右臂] pos=[+0.2977, +0.0562, +0.1881] m, rpy=[  +1.26°,   +2.81°,  -28.74°], gripper=0/255 → ✓ IK成功
[19:24:43.

[19:24:46.493] [左臂] pos=[+0.3351, +0.0270, +0.2236] m, rpy=[  +0.65°,   +2.64°,   +9.91°], gripper=0/255 → ✓ IK成功
[19:24:46.510] [右臂] pos=[+0.2977, +0.0562, +0.1881] m, rpy=[  +1.26°,   +2.81°,  -28.74°], gripper=0/255 → ✓ IK成功
🚀 Fast move (global) speed mode
🚀 Fast move (global) speed mode
[19:24:46.593] [左臂] pos=[+0.3351, +0.0153, +0.2275] m, rpy=[  +0.65°,   +2.64°,   +9.91°], gripper=0/255 → ✓ IK成功
[19:24:46.615] [右臂] pos=[+0.2977, +0.0562, +0.1881] m, rpy=[  +1.26°,   +2.81°,  -28.74°], gripper=0/255 → ✓ IK成功
🚀 Fast move (global) speed mode
🚀 Fast move (global) speed mode
[19:24:46.693] [左臂] pos=[+0.3351, +0.0074, +0.2275] m, rpy=[  +0.65°,   +2.64°,   +9.91°], gripper=0/255 → ✓ IK成功
[19:24:46.717] [右臂] pos=[+0.2977, +0.0562, +0.1881] m, rpy=[  +1.26°,   +2.81°,  -28.74°], gripper=0/255 → ✓ IK成功
🚀 Fast move (global) speed mode
🚀 Fast move (global) speed mode
[19:24:46.793] [左臂] pos=[+0.3351, -0.0004, +0.2275] m, rpy=[  +0.65°,   +2.64°,   +9.91°], gripper=0/255 → ✓ IK成功
[19:24:46.

🚀 Fast move (global) speed mode
[19:24:49.397] [左臂] pos=[+0.3469, -0.0553, +0.2275] m, rpy=[ -20.35°,  +13.64°,  +31.91°], gripper=0/255 → ✓ IK成功
[19:24:49.445] [右臂] pos=[+0.2977, +0.0562, +0.1881] m, rpy=[  +1.26°,   +2.81°,  -27.74°], gripper=0/255 → ✓ IK成功
🚀 Fast move (global) speed mode
🚀 Fast move (global) speed mode
[19:24:49.497] [右臂] pos=[+0.2977, +0.0562, +0.1881] m, rpy=[  +1.26°,   +2.81°,  -27.74°], gripper=0/255 → ✓ IK成功
[19:24:49.519] [左臂] pos=[+0.3469, -0.0553, +0.2275] m, rpy=[ -16.35°,  +13.64°,  +36.91°], gripper=0/255 → ✓ IK成功
🚀 Fast move (global) speed mode
🚀 Fast move (global) speed mode
[19:24:49.597] [左臂] pos=[+0.3469, -0.0553, +0.2275] m, rpy=[  -5.35°,  +11.64°,  +39.91°], gripper=0/255 → ✓ IK成功
[19:24:49.641] [右臂] pos=[+0.2977, +0.0562, +0.1881] m, rpy=[  +1.26°,   +2.81°,  -27.74°], gripper=0/255 → ✓ IK成功
🚀 Fast move (global) speed mode
🚀 Fast move (global) speed mode
[19:24:49.697] [左臂] pos=[+0.3469, -0.0553, +0.2275] m, rpy=[  +3.65°,  +15.64°,  +38.91°], g

[19:24:52.403] [左臂] pos=[+0.3469, -0.0553, +0.2275] m, rpy=[  -3.35°,   +1.64°,  -30.09°], gripper=0/255 → ✓ IK成功
[19:24:52.401] [右臂] pos=[+0.2977, +0.0562, +0.1881] m, rpy=[  +1.26°,   +2.81°,  -27.74°], gripper=0/255 → ✓ IK成功
🚀 Fast move (global) speed mode
🚀 Fast move (global) speed mode
[19:24:52.501] [左臂] pos=[+0.3469, -0.0553, +0.2275] m, rpy=[  -2.35°,   +1.64°,  -30.09°], gripper=0/255 → ✓ IK成功
[19:24:52.521] [右臂] pos=[+0.2977, +0.0562, +0.1881] m, rpy=[  +1.26°,   +2.81°,  -27.74°], gripper=0/255 → ✓ IK成功
🚀 Fast move (global) speed mode
🚀 Fast move (global) speed mode
[19:24:52.601] [左臂] pos=[+0.3469, -0.0553, +0.2275] m, rpy=[  -2.35°,   +1.64°,  -30.09°], gripper=0/255 → ✓ IK成功
[19:24:52.621] [右臂] pos=[+0.2977, +0.0562, +0.1881] m, rpy=[  +1.26°,   +2.81°,  -27.74°], gripper=0/255 → ✓ IK成功
🚀 Fast move (global) speed mode
🚀 Fast move (global) speed mode
[19:24:52.703] [左臂] pos=[+0.3469, -0.0553, +0.2275] m, rpy=[  -2.35°,   +1.64°,  -30.09°], gripper=0/255 → ✓ IK成功
[19:24:52.

[19:24:55.204] [左臂] pos=[+0.3469, -0.0553, +0.2275] m, rpy=[  -2.35°,   +1.64°,  -30.09°], gripper=0/255 → ✓ IK成功
[19:24:55.222] [右臂] pos=[+0.2977, +0.0562, +0.1881] m, rpy=[  +0.26°,   +2.81°,  -36.74°], gripper=0/255 → ✓ IK成功
🚀 Fast move (global) speed mode
🚀 Fast move (global) speed mode
[19:24:55.305] [左臂] pos=[+0.3469, -0.0553, +0.2275] m, rpy=[  -2.35°,   +1.64°,  -30.09°], gripper=0/255 → ✓ IK成功
[19:24:55.307] [右臂] pos=[+0.2977, +0.0562, +0.1881] m, rpy=[  +1.26°,   +2.81°,  -36.74°], gripper=0/255 → ✓ IK成功
🚀 Fast move (global) speed mode
🚀 Fast move (global) speed mode

[右臂] ✗ 客户端断开连接

[左臂] ✗ 客户端断开连接
[右臂] 等待客户端连接...
[左臂] 等待客户端连接...


用户中断程序

正在关闭双臂接收服务...
✓ 右臂TCP服务器端口 46398 已释放
[右臂] 本次连接统计:
  接收数据包: 783
  解码成功帧: 797
  解码统计: {'valid_frames': 797, 'error_frames': 0, 'total_bytes': 17534}
✓ 左臂TCP服务器端口 45511 已释放
[左臂] 本次连接统计:
  接收数据包: 783
  解码成功帧: 797
  解码统计: {'valid_frames': 797, 'error_frames': 0, 'total_bytes': 17534}
✓ TCP端口已释放: 46398, 45511
✓ 双臂接收服务已关闭


## 4. 连接后机械臂回到Home位置

In [23]:
# 在启动服务前，先让两个机械臂回到Home位置
print("="*80)
print("初始化机械臂Home位置...")
print("="*80)

try:
    # 右臂回到Home
    if receiver.right_arm['controller'] and receiver.right_arm['robot']:
        print("\n[右臂] 🏠 移动到Home位置...")
        receiver.right_arm['controller'].move_all_home()

    
    # 左臂回到Home
    if receiver.left_arm['controller'] and receiver.left_arm['robot']:
        print("\n[左臂] 🏠 移动到Home位置...")
        receiver.left_arm['controller'].move_all_home()

    
    print("\n✓ 两个机械臂已初始化到Home位置，准备接收控制信号")
except Exception as e:
    print(f"✗ 初始化失败: {e}")
    import traceback
    traceback.print_exc()

初始化机械臂Home位置...

[右臂] 🏠 移动到Home位置...
🏠 全部舵机同步回中位完成

[左臂] 🏠 移动到Home位置...
🏠 全部舵机同步回中位完成

✓ 两个机械臂已初始化到Home位置，准备接收控制信号


## 使用说明

### 架构概览

```
┌─────────────────────────────────────┐
│      客户端 (电脑)                    │
│  ┌──────────┐  ┌──────────┐        │
│  │ 右手柄    │  │ 左手柄    │        │
│  │ Joycon   │  │ Joycon   │        │
│  └─────┬────┘  └─────┬────┘        │
│        │ TCP           │ TCP        │
│        │ 9091          │ 9092       │
└────────┼───────────────┼────────────┘
         │               │
         ▼               ▼
    ┌─────────────────────────┐
    │   PYNQ (此笔记本)         │
    │                         │
    │ ┌──────┐   ┌──────┐   │
    │ │ 右臂  │   │ 左臂  │   │
    │ │线程   │   │线程   │   │
    │ └──┬───┘   └──┬───┘   │
    │    │          │        │
    │    ▼          ▼        │
    │  ┌─────────────────┐   │
    │  │ 硬件IK求解器     │   │
    │  │0x40000000|0x40010000│
    │  └────┬────────────┘   │
    │       │                │
    │       ▼                │
    │  ┌──────────────────┐  │
    │  │舵机驱动控制      │  │
    │  │/dev/ttyACM0 | /dev/ttyACM1│
    │  └──────────────────┘  │
    └─────────────────────────┘
```

### 主要特性

1. **并行接收**：两个线程同时监听两个端口
2. **独立处理**：各臂独立的IK求解和舵机控制
3. **实时同步**：并行执行双臂运动命令
4. **硬件加速**：使用FPGA硬件求解器加速IK
5. **线程安全**：使用Lock保护共享资源

### 关键参数

| 参数 | 说明 | 默认值 |
|------|------|--------|
| `port_right` | 右臂监听端口 | 9091 |
| `port_left` | 左臂监听端口 | 9092 |
| `servo_port_right` | 右臂舵机串口 | /dev/ttyACM0 |
| `servo_port_left` | 左臂舵机串口 | /dev/ttyACM1 |
| `solver_base_right` | 右臂求解器基地址 | 0x40000000 |
| `solver_base_left` | 左臂求解器基地址 | 0x40010000 |
| `bitstream_path` | FPGA比特流文件 | design_8.bit |

### 运行流程

1. **初始化**：创建两个机械臂、加载比特流、初始化求解器
2. **启动服务**：为两臂启动TCP服务器，监听客户端连接
3. **等待连接**：两个线程分别等待左右手柄连接
4. **接收数据**：并行接收两路Joy-Con控制信号
5. **实时处理**：
   - 解码Joy-Con帧数据
   - 执行IK求解
   - 控制舵机运动
6. **关闭服务**：清理资源，关闭连接